In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
import urllib.request
import io
from io import BytesIO
import json
import requests
from config import nrel_key

In [2]:
# Define URL of target table
url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/20100025/en"

In [3]:
# Create Get request to retrieve JSON data from url
url_json = requests.get(url).json()
# Confirm link returned as value with "object" key from get request
archive_link = url_json["object"]
archive_link

'https://www150.statcan.gc.ca/n1/tbl/csv/20100025-eng.zip'

In [4]:
# Extract content from zip archive
# Define access_url for urllib to open archive_link
access_url = urllib.request.urlopen(archive_link)
# Use ZipFile and BytesIO to read contents from access_url
zf = ZipFile(BytesIO(access_url.read()))
# save CSV file variable of target table from zip archive contents
sales_data = zf.read('20100025.csv')


In [5]:
# Save csv file to pandas DataFrame, decode <class 'bytes'> type to csv format
sales_df = pd.read_csv(io.StringIO(sales_data.decode("utf-8")))
sales_df

,REF_DATE,GEO,DGUID,Zero-Emission Vehicles Fuel Type,Vehicle type,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2017-01,Canada,2016A000011124,All zero-emission vehicles,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277479871,1.1.1.1,3488.0,NaN,NaN,NaN,0
1,2017-01,Canada,2016A000011124,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277485216,1.2.1.1,1664.0,NaN,NaN,NaN,0
2,2017-01,Canada,2016A000011124,Plug-in hybrid electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277490561,1.3.1.1,1824.0,NaN,NaN,NaN,0
3,2017-01,Newfoundland and Labrador,2016A000210,All zero-emission vehicles,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277479872,2.1.1.1,NaN,..,NaN,NaN,0
4,2017-01,Newfoundland and Labrador,2016A000210,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277485217,2.2.1.1,NaN,..,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320695,2021-10,Taloyoak,2016A00056208087,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277490559,5345.2.1.1,NaN,..,NaN,NaN,0
320696,2021-10,Taloyoak,2016A00056208087,Plug-in hybrid electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277495904,5345.3.1.1,NaN,..,NaN,NaN,0
320697,2021-10,"Kitikmeot, Unorganized",2016A00056208098,All zero-emission vehicles,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277485215,5346.1.1.1,NaN,..,NaN,NaN,0
320698,2021-10,"Kitikmeot, Unorganized",2016A00056208098,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277490560,5346.2.1.1,NaN,..,NaN,NaN,0


In [6]:
# Save DataFrame to CSV file in raw_datasets directory
# Define file path to save data
filepath = Path('raw_datasets/EV_sales_table_raw.csv')
# Allow folders and subfolders to be created is needed
filepath.parent.mkdir(parents=True, exist_ok=True)
# Save the file
sales_df.to_csv(filepath)

In [7]:
# Set base URL for Alernative Fuel Stations Call
stations_url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.csv?&limit=all&country=CA&api_key=" + nrel_key

In [8]:
stations_data = pd.read_csv(stations_url, low_memory=False)
stations_data

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,ELEC,Ramada,1319 2nd St W,NaN,Brooks,AB,T1R 1P7,NaN,403-362-6440,E,...,Gratuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,ELEC,Davis Chevrolet,149 E Lake Crescent NE,NaN,Airdrie,AB,T4A 2H9,NaN,403-948-6909,E,...,Gratuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,ELEC,Go Nissan South,1275 101 St SW,NaN,Edmonton,AB,T6X 1A1,NaN,780-463-5700,T,...,Gratuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,ELEC,Don Wheaton Chevrolet,10727 - 82 Ave,NaN,Edmonton,AB,T6E 2B1,NaN,NaN,E,...,Gratuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,ELEC,Gasonic Instruments,8-823 41st Ave NE,NaN,Calgary,AB,T2E 6Y3,NaN,403-276-2201,E,...,Gratuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954,ELEC,City_of_Penticton,171 main street,NaN,Penticton,BC,V2A 5A9,NaN,888-356-8911,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8955,ELEC,13601 Glenoaks blvd,13601 Glenoaks blvd,NaN,Los Angeles,CA,91342,NaN,888-356-8911,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8956,ELEC,6723 Van Nuys blvd,6723 Van Nuys Blvd,NaN,Los Angeles,CA,91405,NaN,888-356-8911,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8957,ELEC,12225 Grenet,12225 Rue Grenet,NaN,Montréal,QC,H4J 2N7,NaN,855-999-8378,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Save DataFrame to CSV file in raw_datasets directory
# Define file path to save data
filepath = Path('raw_datasets/EV_stations_raw.csv')
# Allow folders and subfolders to be created is needed
filepath.parent.mkdir(parents=True, exist_ok=True)
# Save the file
stations_data.to_csv(filepath)